In [1]:
import os

os.environ['OPENAI_API_KEY'] = ''

In [2]:
import json
import openai
import pandas as pd

In [3]:
with open('opendosm.json') as fopen:
    data = json.load(fopen)

In [4]:
def get_embedding(text, model="text-embedding-ada-002"):
    text = text.replace("\n", " ")
    return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']

In [5]:
df = pd.DataFrame(data).T
df = df.reset_index()
df.head()

,index,explanation,metadata,filename
0,https://open.dosm.gov.my/data-catalogue/dosm-p...,"{'bm': {'caveat': 'Bermula 3 Mac 2021, pengumu...",{'url': {'csv': 'https://storage.googleapis.co...,DosmPublicEconomyFuelprice3.csv
1,https://open.dosm.gov.my/data-catalogue/dosm-p...,"{'bm': {'caveat': 'Bermula 3 Mac 2021, pengumu...",{'url': {'csv': 'https://storage.googleapis.co...,DosmPublicEconomyFuelprice1.csv
2,https://open.dosm.gov.my/data-catalogue/dosm-p...,"{'bm': {'caveat': 'Bermula 3 Mac 2021, pengumu...",{'url': {'csv': 'https://storage.googleapis.co...,DosmPublicEconomyFuelprice2.csv
3,https://open.dosm.gov.my/data-catalogue/dosm-p...,{'bm': {'caveat': 'Inflasi teras mengukur peru...,{'url': {'csv': 'https://storage.googleapis.co...,DosmPublicEconomyCpiCore1.csv
4,https://open.dosm.gov.my/data-catalogue/dosm-p...,{'bm': {'caveat': 'Inflasi teras mengukur peru...,{'url': {'csv': 'https://storage.googleapis.co...,DosmPublicEconomyCpiCore3.csv


In [6]:
df_ = pd.read_csv('csv/DosmPublicEconomyFuelprice3.csv')

In [7]:
df_

,series_type,date,ron95,ron97,diesel
0,level,2017-03-30,2.13,2.41,2.11
1,level,2017-04-06,2.16,2.43,2.08
2,level,2017-04-13,2.24,2.52,2.16
3,level,2017-04-20,2.27,2.54,2.21
4,level,2017-04-27,2.21,2.49,2.14
...,...,...,...,...,...
586,change_weekly,2023-02-23,0.00,0.00,0.00
587,change_weekly,2023-03-02,0.00,0.00,0.00
588,change_weekly,2023-03-09,0.00,0.00,0.00
589,change_weekly,2023-03-16,0.00,0.00,0.00


In [8]:
def get_text(row):
    caveat = row['explanation']['en']['caveat']
    methodology = row['explanation']['en']['methodology']
    caveat_ms = row['explanation']['bm']['caveat']
    methodology_ms = row['explanation']['bm']['methodology']
    columns = []
    filename = os.path.join('csv', row['filename'])
    df_ = pd.read_csv(filename)
    for d in row['metadata']['definitions']:
        if 'date' in d['name']:
            unique = None
        else:
            unique = df_[d['name']].unique().tolist()
            if len(unique) > 30:
                unique = None
        s = f"column_name: {d['name']}. description (MS): {d['desc_bm']} description (EN): {d['desc_en']}"
        if unique is not None:
            s = s + f' unique values: {unique}'
        columns.append(s)
    columns = '\n'.join(columns)
    return f'Methodology (EN): {methodology}\nMethodology (MS): {methodology_ms}\nColumns definition:\n{columns}'

df['combined_text'] = df.apply(get_text, axis = 1)
df.head()

,index,explanation,metadata,filename,combined_text
0,https://open.dosm.gov.my/data-catalogue/dosm-p...,"{'bm': {'caveat': 'Bermula 3 Mac 2021, pengumu...",{'url': {'csv': 'https://storage.googleapis.co...,DosmPublicEconomyFuelprice3.csv,Methodology (EN): This dataset presents the we...
1,https://open.dosm.gov.my/data-catalogue/dosm-p...,"{'bm': {'caveat': 'Bermula 3 Mac 2021, pengumu...",{'url': {'csv': 'https://storage.googleapis.co...,DosmPublicEconomyFuelprice1.csv,Methodology (EN): This dataset presents the we...
2,https://open.dosm.gov.my/data-catalogue/dosm-p...,"{'bm': {'caveat': 'Bermula 3 Mac 2021, pengumu...",{'url': {'csv': 'https://storage.googleapis.co...,DosmPublicEconomyFuelprice2.csv,Methodology (EN): This dataset presents the we...
3,https://open.dosm.gov.my/data-catalogue/dosm-p...,{'bm': {'caveat': 'Inflasi teras mengukur peru...,{'url': {'csv': 'https://storage.googleapis.co...,DosmPublicEconomyCpiCore1.csv,Methodology (EN): The index is compiled based ...
4,https://open.dosm.gov.my/data-catalogue/dosm-p...,{'bm': {'caveat': 'Inflasi teras mengukur peru...,{'url': {'csv': 'https://storage.googleapis.co...,DosmPublicEconomyCpiCore3.csv,Methodology (EN): The index is compiled based ...


In [9]:
def get_text(row):
    columns = []
    filename = os.path.join('csv', row['filename'])
    df_ = pd.read_csv(filename)
    for d in row['metadata']['definitions']:
        if 'date' in d['name']:
            unique = None
        else:
            unique = df_[d['name']].unique().tolist()
            if len(unique) > 30:
                unique = None
        s = f"column_name: {d['name']}. description (MS): {d['desc_bm']} description (EN): {d['desc_en']}"
        if unique is not None:
            s = s + f' unique values: {unique}'
        columns.append(s)
    columns = '\n'.join(columns)
    return f'Columns definition:\n{columns}'

df['column_definition'] = df.apply(get_text, axis = 1)
df.head()

,index,explanation,metadata,filename,combined_text,column_definition
0,https://open.dosm.gov.my/data-catalogue/dosm-p...,"{'bm': {'caveat': 'Bermula 3 Mac 2021, pengumu...",{'url': {'csv': 'https://storage.googleapis.co...,DosmPublicEconomyFuelprice3.csv,Methodology (EN): This dataset presents the we...,Columns definition:\ncolumn_name: diesel. desc...
1,https://open.dosm.gov.my/data-catalogue/dosm-p...,"{'bm': {'caveat': 'Bermula 3 Mac 2021, pengumu...",{'url': {'csv': 'https://storage.googleapis.co...,DosmPublicEconomyFuelprice1.csv,Methodology (EN): This dataset presents the we...,Columns definition:\ncolumn_name: ron95. descr...
2,https://open.dosm.gov.my/data-catalogue/dosm-p...,"{'bm': {'caveat': 'Bermula 3 Mac 2021, pengumu...",{'url': {'csv': 'https://storage.googleapis.co...,DosmPublicEconomyFuelprice2.csv,Methodology (EN): This dataset presents the we...,Columns definition:\ncolumn_name: ron97. descr...
3,https://open.dosm.gov.my/data-catalogue/dosm-p...,{'bm': {'caveat': 'Inflasi teras mengukur peru...,{'url': {'csv': 'https://storage.googleapis.co...,DosmPublicEconomyCpiCore1.csv,Methodology (EN): The index is compiled based ...,Columns definition:\ncolumn_name: overall. des...
4,https://open.dosm.gov.my/data-catalogue/dosm-p...,{'bm': {'caveat': 'Inflasi teras mengukur peru...,{'url': {'csv': 'https://storage.googleapis.co...,DosmPublicEconomyCpiCore3.csv,Methodology (EN): The index is compiled based ...,Columns definition:\ncolumn_name: clothing_foo...


In [11]:
print(df.column_definition.iloc[1])

Columns definition:
column_name: ron95. description (MS): [Apung] Harga seliter dalam RM. description (EN): [Float] Price per litre, in RM.
column_name: series_type. description (MS): [Kategori] Harga dalam RM (level), ataupun perubahan harga mingguan dalam RM (change_weekly). description (EN): [categorical] Price in RM (level), or weekly change in RM (change_weekly). unique values: ['level', 'change_weekly']
column_name: date. description (MS): [Tarikh] Tarikh harga berkuatkuasa, dalam format YYYY-MM-DD. description (EN): [Date] The date of effect of the price, in YYYY-MM-DD format
column_name: ron97. description (MS): [Apung] Harga seliter dalam RM. description (EN): [Float] Price per litre, in RM.
column_name: diesel. description (MS): [Apung] Harga seliter dalam RM. description (EN): [Float] Price per litre, in RM.


In [12]:
df['ada_embedding'] = df.combined_text.apply(lambda x: get_embedding(x, model='text-embedding-ada-002'))

In [13]:
df['ada_embedding_column'] = df.column_definition.apply(lambda x: get_embedding(x, model='text-embedding-ada-002'))

In [14]:
df.to_csv('embedded.csv', index=False)